In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import os
from tqdm import tqdm
import json

/home/khaledghalwash/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
2025-11-05 12:50:42.935163: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-05 12:50:43.055108: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762339843.107363   44137 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register 

## 1. Data Collection and Preprocessing

In [ ]:
def extract_landmarks(image_path):
    """Extract hand landmarks from an image using MediaPipe"""
    # Initialize MediaPipe Hands
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=1,
        min_detection_confidence=0.5
    )

    # Read image
    image = cv2.imread(image_path)
    if image is None:
        return None

    # Convert BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Process the image
    results = hands.process(image_rgb)

    # Extract landmarks
    landmarks = []
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            for landmark in hand_landmarks.landmark:
                landmarks.extend([landmark.x, landmark.y, landmark.z])

    hands.close()
    return landmarks if landmarks else None

In [ ]:
def process_dataset(data_dir):
    """Process all images in the dataset directory"""
    data = []
    labels = []

    # Iterate through gesture folders
    for gesture_name in os.listdir(data_dir):
        gesture_dir = os.path.join(data_dir, gesture_name)
        if not os.path.isdir(gesture_dir):
            continue

        print(f"Processing {gesture_name}...")

        # Process each image in the gesture folder
        for image_name in tqdm(os.listdir(gesture_dir)):
            if not image_name.endswith(('.jpg', '.png')):
                continue

            image_path = os.path.join(gesture_dir, image_name)
            landmarks = extract_landmarks(image_path)

            if landmarks:
                data.append(landmarks)
                labels.append(gesture_name)

    return np.array(data), np.array(labels)

## 2. Model Training

In [ ]:
def train_model(X, y):
    """Train and evaluate the Random Forest classifier"""
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )

    # Initialize and train the model
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42
    )

    model.fit(X_train, y_train)

    # Evaluate the model
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)

    # Cross-validation
    cv_scores = cross_val_score(model, X, y, cv=5)

    # Generate predictions
    y_pred = model.predict(X_test)

    return model, {
        'train_score': train_score,
        'test_score': test_score,
        'cv_scores': cv_scores,
        'y_test': y_test,
        'y_pred': y_pred
    }

## 3. Model Evaluation and Visualization

In [ ]:
def plot_confusion_matrix(y_test, y_pred, classes):
    """Plot confusion matrix"""
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes,
                yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

In [ ]:
def plot_feature_importance(model, feature_names):
    """Plot feature importance"""
    importance = model.feature_importances_
    indices = np.argsort(importance)[::-1]

    plt.figure(figsize=(12, 6))
    plt.title('Feature Importance')
    plt.bar(range(len(importance)), importance[indices])
    plt.xticks(range(len(importance)), [feature_names[i] for i in indices], rotation=90)
    plt.tight_layout()
    plt.show()

## 4. Main Execution

In [ ]:
# Set data directory
data_dir = '/home/khaledghalwash/Downloads/depi-project-main/archive(6)/train/train'

print("Using dataset from:", data_dir)
print("\nProcessing gesture folders...\n")

# Check if the directory has data
if not os.path.exists(data_dir) or not any(os.scandir(data_dir)):
    print("\nNo data found in the specified directory!")
else:
    # Process dataset
    X, y = process_dataset(data_dir)
    
    if len(X) == 0:
        print("No valid hand landmarks were extracted from the images.")
    else:
        # Train model
        model, results = train_model(X, y)

        # Print results
        print(f"\nTraining Score: {results['train_score']:.4f}")
        print(f"Testing Score: {results['test_score']:.4f}")
        print(f"Cross-validation Scores: {results['cv_scores']}")
        print(f"Mean CV Score: {np.mean(results['cv_scores']):.4f}")

        # Print classification report
        print("\nClassification Report:")
        print(classification_report(results['y_test'], results['y_pred']))

        # Plot confusion matrix
        plot_confusion_matrix(results['y_test'], results['y_pred'], np.unique(y))

        # Save the model
        os.makedirs('models', exist_ok=True)
        joblib.dump(model, 'models/hand_gesture_classifier.joblib')

        # Save feature names
        feature_names = [f'landmark_{i}' for i in range(X.shape[1])]
        plot_feature_importance(model, feature_names)

Using dataset from: /home/khaledghalwash/Downloads/depi-project-main/archive(6)
Processing train...


100%|██████████| 1/1 [00:00<00:00, 12483.05it/s]


Processing test...


100%|██████████| 1/1 [00:00<00:00, 22550.02it/s]



ValueError: With n_samples=0, test_size=0.3 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

Completed (5/6) *Run tests and start app*

I'll give clear, copyable steps to run the project locally and with Docker, plus quick troubleshooting tips.

## Quick checklist (prerequisites)
- Python 3.8+ (project tested with 3.10/3.11; choose one installed locally)
- pip
- Virtualenv (recommended)
- Webcam (for live demo) or adjust CAMERA_INDEX if using a different device
- Docker (optional, for container run)

## 1) Run locally (recommended for development)
1. Open a terminal and change into the project root:


In [ ]:
cd /home/khaledghalwash/Downloads/depi-project-main



2. Create & activate a virtual environment:


In [ ]:
python -m venv .venv
source .venv/bin/activate



3. Install Python dependencies:


In [ ]:
pip install --upgrade pip
pip install -r requirements.txt



4. Prepare environment file (if present):


In [ ]:
cp .env.example .env
# Edit .env if you need to change CAMERA_INDEX, FRAME_WIDTH, FRAME_HEIGHT, etc.



5. Run the test suite (sanity check):


In [ ]:
pytest

You should see tests pass (in this environment they passed: 4 passed).

6. Start the Streamlit app:


In [ ]:
streamlit run app.py

- Streamlit will print a local URL (usually http://localhost:8501). Open that in your browser.
- To stop the app, press Ctrl+C in the terminal.

If you prefer to run it in the background:


In [ ]:
nohup streamlit run app.py &> streamlit.log &
# then tail -f streamlit.log to watch logs



## 2) Run with Docker (recommended for consistent environment)
1. Build the image from the project root:


In [ ]:
docker build -t hand-gesture-app .



2. Run the container (maps Streamlit port 8501):


In [ ]:
docker run --rm -p 8501:8501 hand-gesture-app

- Visit http://localhost:8501 in the browser.
- Stop container with Ctrl+C (if running foreground) or docker stop <id>.

If you need to expose a local webcam into the container, extra Docker configuration is required (device passthrough or use host network) — tell me if you want that and I’ll show exact docker run flags for your OS.

## 3) Environment variables you can change
- CAMERA_INDEX — index of the camera (default 0)
- FRAME_WIDTH / FRAME_HEIGHT — frame resolution
- MAX_NUM_HANDS, MIN_DETECTION_CONFIDENCE, MIN_TRACKING_CONFIDENCE
You can set them in `.env` or export them before running, e.g.:


In [ ]:
export CAMERA_INDEX=1
streamlit run app.py



## 4) Quick troubleshooting
- Model not found or model is None:
  - Confirm hand_gesture_classifier.joblib exists. The project loads the model from hand_gesture_classifier.joblib.
  - If missing, run the notebook model.ipynb or training scripts to produce a model, or copy a trained `.joblib` into artifacts.
- Camera fails to open:
  - Try different `CAMERA_INDEX` values (0, 1, ...).
  - Verify webcam permissions and that no other process is using it.
  - On headless servers (no camera), you can run but no live video will appear.
- Mediapipe / opencv install issues:
  - On Linux, installing system libs helps (Dockerfile includes apt packages used to avoid issues): libgl1, libsm6, libxext6, libglib2.0-0, ffmpeg.
- Port already in use (8501):
  - Change Streamlit port with environment var:
    ```bash
    export STREAMLIT_SERVER_PORT=8502
    streamlit run app.py
    ```
- If Streamlit logs "Model not loaded" but you expect it to be, check for errors printed at startup (permission, corrupted joblib, version mismatch of scikit-learn). Re-train model with the same scikit-learn version if necessary.

## 5) Useful verification commands
Show model file exists:


In [ ]:
ls -l artifacts/hand_gesture_classifier.joblib



Run tests (already done earlier):


In [ ]:
pytest -q



## 6) Next steps I can do for you
- Start the app in this environment now and stream logs here (confirm and I’ll launch it).
- Help generate a lightweight systemd / process file to run it as a service.
- Add a small health-check endpoint or improve lazy model loading so the app starts even without the model present.

Which next action do you want?